In [61]:
#import packages
import pandas as pd 
from datetime import datetime 
import json

In [ ]:
#define 
file_path = r'C:\Users\snipt\OneDrive\Documents\fhir\Alejandro.json'
df = pd.read_json(file_path)

In [63]:
df

,resourceType,type,entry
0,Bundle,transaction,{'fullUrl': 'urn:uuid:f8d8a950-9d1b-06fa-03d4-...
1,Bundle,transaction,{'fullUrl': 'urn:uuid:5d7fdd5f-4e6d-d8e5-5ce7-...
2,Bundle,transaction,{'fullUrl': 'urn:uuid:459f3302-333c-ef31-eaee-...
3,Bundle,transaction,{'fullUrl': 'urn:uuid:e818a68b-4af5-53b6-6861-...
4,Bundle,transaction,{'fullUrl': 'urn:uuid:164553f3-2a35-2def-e337-...
...,...,...,...
259,Bundle,transaction,{'fullUrl': 'urn:uuid:8b5e6976-182b-05c9-6c8f-...
260,Bundle,transaction,{'fullUrl': 'urn:uuid:1e6bad66-0077-7b19-991c-...
261,Bundle,transaction,{'fullUrl': 'urn:uuid:189c5446-0392-3723-b3ec-...
262,Bundle,transaction,{'fullUrl': 'urn:uuid:3f2b4ba9-984b-6674-6109-...


In [64]:
# Function to extract SNOMED codes from the 'entry' column
def extract_snomed_codes_from_entry(entry):
    # Accessing the 'resource' field inside the entry
    resource = entry.get('resource')

    if resource.get('resourceType') == 'Condition':
        #print(resource)
        #print('Condition resource found')
        coding = resource['code'].get('coding', [])
        #print(coding)
        
        # Extract SNOMED codes and display text
        conditions = [item['display'] for item in coding]
        condition_string = ' '.join(conditions)

        return condition_string
    
    return []

def extract_gender(entry): 
    #access gender 
    resource = entry.get('resource')
    
    if resource.get('resourceType') == 'Patient':
        
        gender = resource.get('gender')
        
        return gender

def extract_birth_date(entry): 
    #access birth date 
    resource = entry.get('resource')
    
    if resource.get('resourceType') == 'Patient':
        
        gender = resource.get('birthDate')
        
        return gender
    
def extract_diagnosis_date(entry):
    resource = entry.get('resource', {})
    
    if resource.get('resourceType') == 'Condition':
        onset_datetime = resource.get('onsetDateTime')

        if onset_datetime:
            # Parse the date or slice to get YYYY-MM-DD
            try:
                onset_date = datetime.fromisoformat(onset_datetime).date()
                return str(onset_date)
            except ValueError:
                return onset_datetime[:10]  # Fallback for malformed dates
    
    return None

def extract_name(entry):
     #access name 
    resource = entry.get('resource')
    
    if resource.get('resourceType') == 'Patient':

        name = resource.get('name')
        official_name = name[0]['given']
        name_string = ' '.join(official_name)
        
        return name_string

# Apply the extraction function to each row in the DataFrame
patient_df = df 
patient_df['name'] = df['entry'].apply(extract_name).fillna(method='ffill')
patient_df['sex'] = df['entry'].apply(extract_gender).fillna(method='ffill')
patient_df['diagnosis_date'] = df['entry'].apply(extract_diagnosis_date)
patient_df['birth_date'] = df['entry'].apply(extract_birth_date).fillna(method='ffill')
patient_df['conditions'] = df['entry'].apply(extract_snomed_codes_from_entry)

In [65]:
# Display the updated DataFrame with SNOMED codes
patient_df = patient_df[patient_df['conditions'].apply(lambda x: bool(x))]
patient_df

,resourceType,type,entry,name,sex,diagnosis_date,birth_date,conditions
58,Bundle,transaction,{'fullUrl': 'urn:uuid:f70501c9-6ce3-49a8-6b5b-...,Alejandro916,male,2014-03-19,2001-01-13,Acute viral pharyngitis (disorder)
202,Bundle,transaction,{'fullUrl': 'urn:uuid:55f14c53-154a-70ef-4cb9-...,Alejandro916,male,2018-12-04,2001-01-13,Viral sinusitis (disorder)
219,Bundle,transaction,{'fullUrl': 'urn:uuid:d26e44ca-27a1-11bb-4ff2-...,Alejandro916,male,2020-02-15,2001-01-13,Acute bacterial sinusitis (disorder)
223,Bundle,transaction,{'fullUrl': 'urn:uuid:b4d9cdca-6379-1c01-aa28-...,Alejandro916,male,2020-02-23,2001-01-13,Sputum finding (finding)
224,Bundle,transaction,{'fullUrl': 'urn:uuid:798b678c-ba57-1dac-f6cc-...,Alejandro916,male,2020-02-23,2001-01-13,Fever (finding)
225,Bundle,transaction,{'fullUrl': 'urn:uuid:21eaebfe-995e-c462-ab16-...,Alejandro916,male,2020-02-23,2001-01-13,Loss of taste (finding)
226,Bundle,transaction,{'fullUrl': 'urn:uuid:1507922d-912b-7712-ce48-...,Alejandro916,male,2020-02-23,2001-01-13,Suspected COVID-19
227,Bundle,transaction,{'fullUrl': 'urn:uuid:d912349e-c5d1-26cb-0b4f-...,Alejandro916,male,2020-02-23,2001-01-13,COVID-19


In [66]:
#back calculate the age
patient_df['birth_date'] = pd.to_datetime(patient_df['birth_date'])
patient_df['diagnosis_date'] = pd.to_datetime(patient_df['diagnosis_date'])
patient_df['age_at_diagnosis'] = (patient_df['diagnosis_date'] - patient_df['birth_date']).dt.days // 365
patient_df

C:\Users\snipt\AppData\Local\Temp\ipykernel_25404\4241226509.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patient_df['birth_date'] = pd.to_datetime(patient_df['birth_date'])
C:\Users\snipt\AppData\Local\Temp\ipykernel_25404\4241226509.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patient_df['diagnosis_date'] = pd.to_datetime(patient_df['diagnosis_date'])
C:\Users\snipt\AppData\Local\Temp\ipykernel_25404\4241226509.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

,resourceType,type,entry,name,sex,diagnosis_date,birth_date,conditions,age_at_diagnosis
58,Bundle,transaction,{'fullUrl': 'urn:uuid:f70501c9-6ce3-49a8-6b5b-...,Alejandro916,male,2014-03-19,2001-01-13,Acute viral pharyngitis (disorder),13
202,Bundle,transaction,{'fullUrl': 'urn:uuid:55f14c53-154a-70ef-4cb9-...,Alejandro916,male,2018-12-04,2001-01-13,Viral sinusitis (disorder),17
219,Bundle,transaction,{'fullUrl': 'urn:uuid:d26e44ca-27a1-11bb-4ff2-...,Alejandro916,male,2020-02-15,2001-01-13,Acute bacterial sinusitis (disorder),19
223,Bundle,transaction,{'fullUrl': 'urn:uuid:b4d9cdca-6379-1c01-aa28-...,Alejandro916,male,2020-02-23,2001-01-13,Sputum finding (finding),19
224,Bundle,transaction,{'fullUrl': 'urn:uuid:798b678c-ba57-1dac-f6cc-...,Alejandro916,male,2020-02-23,2001-01-13,Fever (finding),19
225,Bundle,transaction,{'fullUrl': 'urn:uuid:21eaebfe-995e-c462-ab16-...,Alejandro916,male,2020-02-23,2001-01-13,Loss of taste (finding),19
226,Bundle,transaction,{'fullUrl': 'urn:uuid:1507922d-912b-7712-ce48-...,Alejandro916,male,2020-02-23,2001-01-13,Suspected COVID-19,19
227,Bundle,transaction,{'fullUrl': 'urn:uuid:d912349e-c5d1-26cb-0b4f-...,Alejandro916,male,2020-02-23,2001-01-13,COVID-19,19


In [67]:
import re 

def remove_parentheses(text):
    return re.sub(r'\(.*?\)', '', text)

# Apply the function to the 'conditions' column
patient_df['clean_conditions'] = patient_df['conditions'].apply(remove_parentheses)
patient_df


C:\Users\snipt\AppData\Local\Temp\ipykernel_25404\3583738640.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patient_df['clean_conditions'] = patient_df['conditions'].apply(remove_parentheses)


,resourceType,type,entry,name,sex,diagnosis_date,birth_date,conditions,age_at_diagnosis,clean_conditions
58,Bundle,transaction,{'fullUrl': 'urn:uuid:f70501c9-6ce3-49a8-6b5b-...,Alejandro916,male,2014-03-19,2001-01-13,Acute viral pharyngitis (disorder),13,Acute viral pharyngitis
202,Bundle,transaction,{'fullUrl': 'urn:uuid:55f14c53-154a-70ef-4cb9-...,Alejandro916,male,2018-12-04,2001-01-13,Viral sinusitis (disorder),17,Viral sinusitis
219,Bundle,transaction,{'fullUrl': 'urn:uuid:d26e44ca-27a1-11bb-4ff2-...,Alejandro916,male,2020-02-15,2001-01-13,Acute bacterial sinusitis (disorder),19,Acute bacterial sinusitis
223,Bundle,transaction,{'fullUrl': 'urn:uuid:b4d9cdca-6379-1c01-aa28-...,Alejandro916,male,2020-02-23,2001-01-13,Sputum finding (finding),19,Sputum finding
224,Bundle,transaction,{'fullUrl': 'urn:uuid:798b678c-ba57-1dac-f6cc-...,Alejandro916,male,2020-02-23,2001-01-13,Fever (finding),19,Fever
225,Bundle,transaction,{'fullUrl': 'urn:uuid:21eaebfe-995e-c462-ab16-...,Alejandro916,male,2020-02-23,2001-01-13,Loss of taste (finding),19,Loss of taste
226,Bundle,transaction,{'fullUrl': 'urn:uuid:1507922d-912b-7712-ce48-...,Alejandro916,male,2020-02-23,2001-01-13,Suspected COVID-19,19,Suspected COVID-19
227,Bundle,transaction,{'fullUrl': 'urn:uuid:d912349e-c5d1-26cb-0b4f-...,Alejandro916,male,2020-02-23,2001-01-13,COVID-19,19,COVID-19


In [68]:
# Convert the relevant patientnt columns into a string for each row
patient_summary = ""

# Iterate over each row in the DataFrame
for index, row in patient_df.iterrows():
    patient_string = (
        f"Patient Name: {row['name']}, "
        f"Sex: {row['sex']}, "
        f"Diagnosis Date: {row['diagnosis_date']}, "
        f"Condition: {row['conditions']}, "
        f"Age at Diagnosis: {row['age_at_diagnosis']}\n"
    )
    # Append the patient string to the overall summary
    patient_summary += patient_string

# After the loop, patient_summary contains all patient strings combined
print(patient_summary)

Patient Name: Alejandro916, Sex: male, Diagnosis Date: 2014-03-19 00:00:00, Condition: Acute viral pharyngitis (disorder), Age at Diagnosis: 13
Patient Name: Alejandro916, Sex: male, Diagnosis Date: 2018-12-04 00:00:00, Condition: Viral sinusitis (disorder), Age at Diagnosis: 17
Patient Name: Alejandro916, Sex: male, Diagnosis Date: 2020-02-15 00:00:00, Condition: Acute bacterial sinusitis (disorder), Age at Diagnosis: 19
Patient Name: Alejandro916, Sex: male, Diagnosis Date: 2020-02-23 00:00:00, Condition: Sputum finding (finding), Age at Diagnosis: 19
Patient Name: Alejandro916, Sex: male, Diagnosis Date: 2020-02-23 00:00:00, Condition: Fever (finding), Age at Diagnosis: 19
Patient Name: Alejandro916, Sex: male, Diagnosis Date: 2020-02-23 00:00:00, Condition: Loss of taste (finding), Age at Diagnosis: 19
Patient Name: Alejandro916, Sex: male, Diagnosis Date: 2020-02-23 00:00:00, Condition: Suspected COVID-19, Age at Diagnosis: 19
Patient Name: Alejandro916, Sex: male, Diagnosis Date:

In [69]:
# Extract the 'conditions' column and combine them into a single string
conditions_str = ' OR '.join(patient_df['clean_conditions'].values)

# Output the combined conditions string
print(conditions_str)

Acute viral pharyngitis  OR Viral sinusitis  OR Acute bacterial sinusitis  OR Sputum finding  OR Fever  OR Loss of taste  OR Suspected COVID-19 OR COVID-19


In [70]:
import requests
import json 
from pandas import json_normalize

In [71]:
url = "https://clinicaltrials.gov/api/v2/studies"

headers = {
    "Content-Type": "application/json", 
}

params = {
    "query.cond": conditions_str, 
    "filter.overallStatus": ["RECRUITING"], 
}

#write function to get the nctID, Official Title, Eligibility Criteria, Disease Characteristics
#def extract_clinical_trial_information(dataframe): 


    
    #return None

# Make the GET request
response = requests.get(url, headers=headers, params=params)

# Print response
if response.status_code == 200:
    print("Success:", response.json())
    print(response.request.url) 
    response_json = response.json()
    clinical_df = json_normalize(response_json.get('studies', []))
else:
    print(f"Error {response.status_code}: {response.text}")


#final_clinical_df = extract_clinical_trial_information(clinical_trial)

Success: {'studies': [{'protocolSection': {'identificationModule': {'nctId': 'NCT06684431', 'orgStudyIdInfo': {'id': 'CCHF-NP-1'}, 'organization': {'fullName': 'Karolinska Institutet', 'class': 'OTHER'}, 'briefTitle': 'Study to Evaluate Safety and Immunogenicity of DNA Vaccine N-pVAX1 Against Crimean Congo Hemorrhagic Fever', 'officialTitle': 'A Phase I Study to Evaluate Safety and Immunogenicity of DNA Vaccine N-pVAX1 Against Crimean Congo Hemorrhagic Fever', 'acronym': 'CCHF-NP-1'}, 'statusModule': {'statusVerifiedDate': '2024-11', 'overallStatus': 'RECRUITING', 'expandedAccessInfo': {'hasExpandedAccess': False}, 'startDateStruct': {'date': '2024-11-12', 'type': 'ACTUAL'}, 'primaryCompletionDateStruct': {'date': '2026-12-31', 'type': 'ESTIMATED'}, 'completionDateStruct': {'date': '2026-12-31', 'type': 'ESTIMATED'}, 'studyFirstSubmitDate': '2024-11-11', 'studyFirstSubmitQcDate': '2024-11-11', 'studyFirstPostDateStruct': {'date': '2024-11-12', 'type': 'ACTUAL'}, 'lastUpdateSubmitDate':

In [72]:
#take the previosu json response and get the nctID, Official Title, Eligibility Criteria, Disease Characteristics
clinical_df = clinical_df[['protocolSection.identificationModule.nctId', 'protocolSection.identificationModule.briefTitle', 'protocolSection.identificationModule.officialTitle', 'protocolSection.descriptionModule.briefSummary', 'protocolSection.descriptionModule.detailedDescription', 'protocolSection.conditionsModule.conditions', 'protocolSection.conditionsModule.keywords', 'protocolSection.eligibilityModule.eligibilityCriteria', 'protocolSection.eligibilityModule.sex', 'protocolSection.eligibilityModule.minimumAge']]
clinical_df

,protocolSection.identificationModule.nctId,protocolSection.identificationModule.briefTitle,protocolSection.identificationModule.officialTitle,protocolSection.descriptionModule.briefSummary,protocolSection.descriptionModule.detailedDescription,protocolSection.conditionsModule.conditions,protocolSection.conditionsModule.keywords,protocolSection.eligibilityModule.eligibilityCriteria,protocolSection.eligibilityModule.sex,protocolSection.eligibilityModule.minimumAge
0,NCT06684431,Study to Evaluate Safety and Immunogenicity of...,A Phase I Study to Evaluate Safety and Immunog...,This First-in-human dose-escalation vaccine ph...,This First-in-human dose-escalation vaccine ph...,[Crimean Congo Hemorrhagic Fever],"[N-pVAX1, Phase I, Dose-escalation]",Inclusion Criteria:\n\n1. Men and women betwee...,ALL,18 Years
1,NCT05735782,Clinical and Biological Characterization of Po...,"Association of Symptoms, Clinical and Biochemi...",The goal of this observational study is to des...,Two groups of cases and one group of controls ...,[Long COVID],"[Post-Acute COVID-19 Syndrome, Long COVID, Pos...",Two groups of cases and one group of controls ...,ALL,18 Years
2,NCT05505682,Impact of Project Wolbachia - Singapore on Den...,Assessing the Efficacy of Male Wolbachia-infec...,The study is a cluster-randomised controlled t...,"The study is designed as a parallel, two-arm, ...",[Dengue],NaN,Inclusion criteria:\n\n* Dengue-suspected pati...,ALL,NaN
3,NCT05787431,Levels of Physical Activity After SARS-CoV-2 I...,Levels of Physical Activity After SARS-CoV-2 I...,Coronavirus infection has an impact on several...,"Coronavirus identified in 2019, SARS-CoV-2 (CO...",[COVID-19 (Coronavirus Disease 2019)],"[Physical Activity, exercise, public health, P...",Inclusion Criteria:\n\nCOVID-19 recovered indi...,ALL,18 Years
4,NCT05808231,Effectiveness and Safety of Quinine Sulfate as...,"An aDaptive, multicEnter, rAndomized, Open-Lab...","This is a multicenter, randomized, open-label,...","This is an adaptive, multicenter, randomized, ...",[COVID-19],"[COVID-19, Quinine Sulfate]",Inclusion Criteria:\n\n* Male and female patie...,ALL,18 Years
5,NCT03842982,Hyperthermic Intraperitoneal Chemotherapy (HIP...,Phase III Randomized Clinical Trial Evaluating...,"This is a phase III, multicenter, intervention...",The primary objective of this study is to asse...,"[Ovary Neoplasms, Ovarian Cancer, Ovarian Carc...","[Ovarian cancer, Hyperthermic intraperitoneal ...",Inclusion Criteria:\n\nPre-eligibility criteri...,FEMALE,18 Years
6,NCT05525182,Efficacy and Safety of ES16001 in Patients Wit...,Efficacy and Safety of ES16001 in Patients Wit...,"This is phase II/III, randomized, parallel-gro...",After obtaining the consent and confirming eli...,[COVID-19],NaN,Inclusion Criteria:\n\n1. Those with full unde...,ALL,19 Years
7,NCT05539482,Community-Based Health Education Programs for ...,"A Multicenter, Cluster-Randomized, Active-Cont...",This study evaluates the community-based healt...,This health education program is based on a Co...,[COVID-19],"[CBPR, health education, vaccination rate, ear...",For participants recruited by NGOs and compani...,ALL,12 Years
8,NCT05582382,Validation of Prognostic Clinical Risk Scores ...,Validation of Prognostic Clinical Risk Scores ...,Background Severe acute respiratory syndrome c...,"Background:\n\nSince December 2019, when sever...",[COVID-19],"[hospitalization, COVID-19, clinical risk scor...",Inclusion Criteria:\n\n* All consecutive patie...,ALL,16 Years
9,NCT04910282,Impact of COVID-19 on Quality of Life of Senio...,Impact of COVID-19 on Quality of Life of Senio...,The ophthalmology clientele is vulnerable in t...,BACKGROUND AND RATIONALE\n\nCOVID-19 is caused...,"[Covid19, Eye Diseases]","[Eye disease patients, Psychological stressors...",Inclusion Criteria:\n\n* Patients diagnosed wi...,ALL,65 Years


In [73]:
# Initialize an empty string to store the combined output
trial_list = ""

# Loop through each row in the DataFrame and concatenate the information
for index, row in clinical_df.iterrows():
    # Concatenate relevant columns into a single string for each row
    trial_info = (
        f"NCT ID: {row['protocolSection.identificationModule.nctId']}, "
        #f"Brief Title: {row['protocolSection.identificationModule.briefTitle']}, "
        f"Official Title: {row['protocolSection.identificationModule.officialTitle']}, "
        #f"Brief Summary: {row['protocolSection.descriptionModule.briefSummary']}, "
        #f"Detailed Description: {row['protocolSection.descriptionModule.detailedDescription']}, "
        #f"Conditions: {', '.join(row['protocolSection.conditionsModule.conditions'])}, "
        #f"Keywords: {', '.join(row['protocolSection.conditionsModule.keywords'])}, "
        #f"Eligibility Criteria: {row['protocolSection.eligibilityModule.eligibilityCriteria']}, "
        f"Sex: {row['protocolSection.eligibilityModule.sex']}, "
        f"Minimum Age: {row['protocolSection.eligibilityModule.minimumAge']}\n\n"
    )
    
    # Append this patient string to the combined string
    trial_list += trial_info

# Print the combined string
print(trial_list)

NCT ID: NCT06684431, Official Title: A Phase I Study to Evaluate Safety and Immunogenicity of DNA Vaccine N-pVAX1 Against Crimean Congo Hemorrhagic Fever, Sex: ALL, Minimum Age: 18 Years

NCT ID: NCT05735782, Official Title: Association of Symptoms, Clinical and Biochemical Characteristics in Post COVID-19 Syndrome: a Case-control Study, Sex: ALL, Minimum Age: 18 Years

NCT ID: NCT05505682, Official Title: Assessing the Efficacy of Male Wolbachia-infected Mosquito Deployments to Reduce Dengue Incidence in Singapore: a Cluster Randomized Controlled Trial, Sex: ALL, Minimum Age: nan

NCT ID: NCT05787431, Official Title: Levels of Physical Activity After SARS-CoV-2 Infection in Greek Adults: the Role of Pre-illness Vaccination and the Number of Reinfections., Sex: ALL, Minimum Age: 18 Years

NCT ID: NCT05808231, Official Title: An aDaptive, multicEnter, rAndomized, Open-Label, Controlled Trial to Assess Effectiveness and Safety of Quinine Sulfate for COVID-19 in Hospitalized Adults, Sex: 

In [74]:
import openai 

In [ ]:
# Set your API key
openai.api_key = ""

# List of prompts to query the LLM
prompts = [
    f"Summarize this patient's medical history: '{patient_summary}'", 
    f"I have provided a list of clinical trials with eligibility and inclusion information. Please summarize trials and identify the characteristics of a patient that would be a good candidate for these trials '{trial_list}'", 
    f"""
Please evaluate whether the patient described in "{patient_summary}" would be eligible for each clinical trial listed in {trial_list}. For each trial, provide:
1. A **Match Score** from 0% to 100%.
2. A **Rationale** explaining why the patient is or isn’t eligible for the trial.

Ensure that each trial's response is clearly organized with the following labels:
- **NCT ID**: [The trial's NCT ID]
- **Match Score**: [The eligibility score, e.g., 0%, 25%, 50%, 75%, 100%]
- **Rationale**: [A brief explanation of why the patient is or isn't eligible for this trial]
"""
]

# Store responses
responses = []

# Query the chat model for each prompt
for prompt in prompts:
    response = openai.ChatCompletion.create(
        model="gpt-4",  # Specify your chat model
        messages=[
            {"role": "system", "content": "You are a clinical research assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=4000
    )
    responses.append(response.choices[0].message["content"].strip())  # Save the response

# Print all responses
for i, (prompt, response) in enumerate(zip(prompts, responses), 1):
    #print(f"Prompt {i}: {prompt}")
    print(f"Response {i}: {response}\n")

Response 1: Patient Alejandro916, a male, was diagnosed with Acute viral pharyngitis at the age of 13 in 2014. In 2018, at age 17, he was diagnosed with Viral sinusitis. In 2020, at age 19, Alejandro916 was diagnosed with a series of medical conditions beginning with Acute bacterial sinusitis, followed by a sputum finding, fever, loss of taste, a suspicion of COVID-19 before finally being diagnosed with COVID-19.

Response 2: 1) NCT06684431 - This trial is a phase I study evaluating the safety and immunogenicity of the DNA vaccine N-pVAX1 against Crimean Congo Hemorrhagic Fever. Open to all individuals aged 18 and above.

2) NCT05735782 - It is a case-control study investigating the association of symptoms, clinical, and biochemical characteristics in post COVID-19 syndrome. All individuals 18 years and older are eligible.

3) NCT05505682 - The efficacy of male Wolbachia-infected mosquito deployments in reducing the incidence of dengue in Singapore is being assessed in this cluster ran